In [2]:
# Requires GenePattern Notebook: pip install genepattern-notebook
import gp
import genepattern

# Username and password removed for security reasons.
genepattern.display(genepattern.session.register("https://cloud.genepattern.org/gp", "", ""))

GPAuthWidget()

In [3]:
import gp, gp.data
import pandas as pd
from glob import glob
import re
import os
import time
#import wget
from io import StringIO

#enter your GP info early to access DiSCoVER
print("Welcome to the DataFrame Generator")
username = input("Enter GP username :")
password = input("Enter GP password :")

#access the GP server, import the select .gct file to parse  NOTE: needs to be revised
gpserver = gp.GPServer('https://cloud.genepattern.org/gp', username, password)
print("Select the GCT file to process:")
df = pd.read_csv('https://datasets.genepattern.org/data/ccmi_tutorial/2018-09-20/BRCA_HUGO_symbols.preprocessed.gct',
                  header=2,sep='\t').drop('Description',axis=1)

names = df.drop('Name',axis=1).columns

#this will get the discover module loading   
module = gp.GPTask(gpserver, "DiSCoVER")
module.param_load()

#directory for the 1st wave of CSV files
print("a folder named 'CSVraw' will now be added to your directory")
print("the resulting CSV files will be added to 'CSVraw'")
dir = "/home/jovyan/CSVraw"
try:
  os.mkdir(dir)
  print("a folder named 'CSVraw' will now be added to your directory")
except:
  pass


if len(os.listdir(dir) ) == 0:
    print("Directory is empty")
    for name in names:
        df_new = df[['Name',name]].to_csv("/home/jovyan/CSVraw/"+name+'.csv',header=True,index=True)
else:    
    print("Directory is not empty")
    
#access all the new CSV files
cancer_and_normal = glob("CSVraw/*.csv")
cancer_and_normal.sort()
print (str((len(cancer_and_normal)/2)))
#print(os.getcwd())

# create the directory fot the cvs results
dir = "/home/jovyan/CSVresults"
try:
    os.mkdir(dir)
except:
    pass
        
numerito = (len(os.listdir(dir)))*2
numerito2 = numerito/2
print("CVS Results Created: ", numerito2)

#this will access every two files that have a certain        
for files in cancer_and_normal:
    if (numerito>0):
        numerito = numerito-1
        print('Skipping one file.')
    else:
        directory = str (files)
        namecell =(directory[7:22:1])
        typecell = (directory[20:22:1])
        print(files)

        if typecell == "01" :
            cancer_sample_name = namecell
            cancer_sample_dir = directory

        elif typecell == "11" :
            normal_sample_name = namecell
            normal_sample_dir = directory
            job_spec = module.make_job_spec()
            uploaded_gene_file = gpserver.upload_file(cancer_sample_name, cancer_sample_dir)
            print(uploaded_gene_file)
            uploaded_control_file = gpserver.upload_file(normal_sample_name, normal_sample_dir )
            job_spec.set_parameter("gene_expression", uploaded_gene_file.get_url()) 
            job_spec.set_parameter("use_custom_control", "True")
            job_spec.set_parameter("control", uploaded_control_file.get_url())

            job = gpserver.run_job(job_spec, False)
            job.is_finished()  # Quaries the server and returns True if the job is complete, False otherwise
            job.get_info()  # Returns a brief description of the job's current state

            while not job.is_finished():
                print("Still running...")
                time.sleep(120)
            numerito2 = numerito2+1
            print("CVS Results Created: ", numerito2)

            print(job.is_finished())
            output_list = job.get_output_files()  # Get a list of output files

            for file in output_list:  # Loop through each output file
                if file.get_url().endswith('.txt'):
                    print('skipping',file)
                elif file.get_url().endswith('.csv'):
                    print( file.get_url() )  # Print the URL to the file
        #             GP_csv = file.get_url()
        #             temp = GP_csv.split('/')
        #             # programatically access that job to open the file
        #             GP_csv = eval('job'+temp[5]+'.get_file("'+temp[6]+'")')
        #             print('job'+temp[5]+'.get_file("'+temp[6]+'")')
                    drug_df = pd.read_csv(StringIO(file.read()),index_col=0)
                    print(drug_df.head())

                    #if (len(os.listdir(dir) ) != len(cancer_and_normal)/2):
                    print('==> About to write file',dir+cancer_sample_name+'_DiSCoVER_results.csv')
                    drug_df.to_csv(dir+'/'+cancer_sample_name+'_DiSCoVER_results.csv')
                else:
                    print('Unknown file type:',file)

    #             wget.download(file,'/home/jovyan/CSVresults')
                # if the folder has the necessary number of files...

            

Welcome to the DataFrame Generator
Enter GP username :HugoElcabezas
Enter GP password :HuGo10DeMaYo
Select the GCT file to process:
a folder named 'CSVraw' will now be added to your directory
the resulting CSV files will be added to 'CSVraw'
Directory is not empty
20.0
CVS Results Created:  20.0
Skipping one file.
Skipping one file.
Skipping one file.
Skipping one file.
Skipping one file.
Skipping one file.
Skipping one file.
Skipping one file.
Skipping one file.
Skipping one file.
Skipping one file.
Skipping one file.
Skipping one file.
Skipping one file.
Skipping one file.
Skipping one file.
Skipping one file.
Skipping one file.
Skipping one file.
Skipping one file.
Skipping one file.
Skipping one file.
Skipping one file.
Skipping one file.
Skipping one file.
Skipping one file.
Skipping one file.
Skipping one file.
Skipping one file.
Skipping one file.
Skipping one file.
Skipping one file.
Skipping one file.
Skipping one file.
Skipping one file.
Skipping one file.
Skipping one file.


In [4]:
#numFiles carries discover output csv files
numFiles = []
all_paths = glob("CSVresults/*.csv")
all_paths

#df_preList carries dataframes
df_preList = []
score_df = pd.DataFrame([])

#this for loop will fill up the list numFiles with all the CSV files
for filename in all_paths:
    numFiles.append(filename)
    
# this for loop will take every CSV in the list numFiles and create their corresponding dataframe
# these dataframes are then added to the list df_preList
for i in range(len(all_paths)):
        df_csv = pd.read_csv(numFiles[i])
        df_preList.append(df_csv)
        
#this for loop will extract the "score" column from every csv dataframe and 
#add them to an existing one that only carries name, moa, and the first score
counter = 0
for i in range(len(all_paths)):
    if counter == 0:
        df_final = df_preList[counter]
        df_final = df_final.set_index("drug")
        df_final.index.names = ['NAME']
        tempstring = str (numFiles[i])
        tempstring =(tempstring[-36::1])
        tempstring =(tempstring[0:15:1])
        print(tempstring)
        
        new_order = ['moa','score']
        df_final=df_final[new_order]
        
        df_final=df_final.rename(columns = {'score':tempstring,'moa':'Description'})
        
    else:
        tempstring = str (numFiles[i])
        tempstring =(tempstring[-36::1])
        tempstring =(tempstring[0:15:1])
        print(tempstring)
            
        col_name = tempstring
        temp = df_preList[i]
        temp = temp.set_index("drug")
        df_final[col_name] = temp["score"]
        
    counter += 1
    
def standarize_string(what):
    ascii_8bit = re.sub(r'[^ -~]', '', what).lower() # keeping only 8-bit ascii, making lowercase (http://www.catonmat.net/blog/my-favorite-regex/)
    return ascii_8bit
    #return re.sub(r'[ _-]', '', ascii_8bit) # remove space, underscore, and dash (" _-")
    
for index, row in df_final.iterrows():
    
    df_final.loc[index,'Description'] = standarize_string(str (row['Description']))
    
df_final = df_final.sort_values(by=['NAME'])
df_final
gp.data.write_gct(df_final, file_path='./df_test7.gct')
print("DataFrame Successfully Created!")

TCGA-BH-A0B3-01
TCGA-BH-A0BC-01
TCGA-A7-A0CH-01
TCGA-BH-A0B7-01
TCGA-A7-A13G-01
TCGA-AC-A23H-01
TCGA-BH-A0B5-01
TCGA-A7-A13E-01
TCGA-AC-A2FM-01
TCGA-A7-A0D9-01
TCGA-BH-A0AZ-01
TCGA-BH-A0BA-01
TCGA-A7-A0CE-01
TCGA-BH-A0AU-01
TCGA-BH-A0B8-01
TCGA-BH-A0AY-01
TCGA-AC-A2FF-01
TCGA-A7-A13F-01
TCGA-A7-A0DB-01
TCGA-AC-A2FB-01
DataFrame Successfully Created!
